In [4]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import chi2_contingency 
from pipelines import preprocessor, preprocessor2
import pickle
import joblib

In [5]:
train= pd.read_csv("Datasets/train.csv")

<IPython.core.display.Javascript object>

In [87]:
X=train.iloc[:,:-1]
y=train.iloc[:, -1]

In [88]:
X.shape

(18506, 20)

In [89]:
X.head(2)

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz
0,tour_id1hffseyw,ITALY,45-64,With Children,0.0,2.0,Visiting Friends and Relatives,Beach Tourism,"Friends, relatives",Package Tour,Yes,Yes,Yes,Yes,No,No,No,0,7,Yes
1,tour_idnacd7zag,UNITED KINGDOM,25-44,With Spouse,1.0,1.0,Leisure and Holidays,Wildlife Tourism,"Travel agent, tour operator",Package Tour,Yes,Yes,Yes,Yes,No,No,No,0,7,Yes


In [91]:
col_drop=[]

for col in X.columns:
    
    ct=pd.crosstab(y, X[col])
    chi2, P, _ , _ = chi2_contingency(ct)
    
    if P >= 0.05:
        col_drop.append(col)
    elif chi2 <= 1500:
            col_drop.append(col)
        
    print(f"Column: {col}\nChi_Test: {chi2}\nP_Value: {P}\n...............")

<IPython.core.display.Javascript object>

Column: Tour_ID
Chi_Test: 92530.0
P_Value: 0.4947449902995666
...............


<IPython.core.display.Javascript object>

Column: country
Chi_Test: 9779.033678192025
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

...

Column: night_mainland
Chi_Test: 5614.015301227679
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

Column: night_zanzibar
Chi_Test: 2652.1882265219597
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

Column: first_trip_tz
Chi_Test: 2229.180451768042
P_Value: 0.0
...............


In [92]:
col_drop

['Tour_ID', 'total_male']

In [93]:
X.drop(columns=col_drop, inplace=True)
X.shape

(18506, 18)

In [94]:
X= preprocessor(X)
X

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [76]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y=le.fit_transform(y)
mapping= dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping)

{'High Cost': 0, 'Higher Cost': 1, 'Highest Cost': 2, 'Low Cost': 3, 'Lower Cost': 4, 'Normal Cost': 5}


In [77]:
mapping

{'High Cost': 0,
 'Higher Cost': 1,
 'Highest Cost': 2,
 'Low Cost': 3,
 'Lower Cost': 4,
 'Normal Cost': 5}

In [95]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, shuffle=True, stratify= y, random_state=1)

print(f"X_train: {X_train.shape}\nX_test: {X_test.shape}\ny_train: {y_train.shape}\ny_test: {y_test.shape}")

X_train: (14804, 187)
X_test: (3702, 187)
y_train: (14804,)
y_test: (3702,)


In [96]:
n_features= X.shape[1]
num_classes= len(le.classes_)

model= Sequential()
model.add(Dense(32, input_dim=n_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

epochs=30
learning_rate= 0.01
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 32)                6016      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_9 (Dense)             (None, 32)                1056      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 128)              

In [98]:
model.fit(X_train, y_train, epochs= epochs, batch_size=64, verbose=1)

Epoch 1/30


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "C:\Users\User\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\User\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\User\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\User\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\User\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    File "C:\Users\User\ana

In [ ]:
loss, accuracy= model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy Score: {accuracy }\nLoss: {loss}")